In [1]:
library(ggplot2)
library(data.table)
max_ac_df <- readRDS("hour18_data.Rds")

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang


In [2]:
# max_ac_df$altitude <- 500 * round(max_ac_df$altitude/500)

In [3]:
# max_ac_df$altitude <- as.factor(max_ac_df$altitude)

In [4]:
dir.create("hour18", showWarnings = F)
setwd("hour18")


In [5]:
getwd()

[1] "/Users/naveensathiyanathan/Desktop/Coding/Old/ATC_Analysis_2019_04_01/Refedined_Landing/hour18"

In [6]:
for(i in 0:4) {
    dir.create(as.character(i), showWarnings = F)
}

In [7]:
idx <- which(max_ac_df$aircraft == max_ac_df$destination)
max_ac_df$aircraft[idx] <- max_ac_df$aircraft[idx - 1]
max_ac_df$azimuth[is.na(max_ac_df$azimuth)] <- 0
head(max_ac_df)

last_destination,aircraft,ts,V1,V2.x,lat.x,lon.x,azimuth,altitude,ground_speed,⋯,V12.y,V13,V14,near_destination,landed,landed_tot,entered,landed_or_entered,num_id,id
<chr>,<chr>,<int>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<int>,⋯,<chr>,<chr>,<chr>,<lgl>,<lgl>,<int>,<lgl>,<lgl>,<int>,<chr>
,001915,1558997679,1558997673,20aba14c,41.0922,-73.7795,0,2825,0,⋯,NA,NA,NA,FALSE,FALSE,0,FALSE,FALSE,0,001915_0
,001915,1558997700,1558997700,20aba14c,41.0849,-73.8232,0,4104,0,⋯,NA,NA,NA,FALSE,FALSE,0,FALSE,FALSE,0,001915_0
,001915,1558997712,1558997717,20aba14c,41.0769,-73.8534,0,5500,0,⋯,NA,NA,NA,FALSE,FALSE,0,FALSE,FALSE,0,001915_0
,001915,1558997727,1558997726,20aba14c,41.0646,-73.8861,0,6075,0,⋯,NA,NA,NA,FALSE,FALSE,0,FALSE,FALSE,0,001915_0
,001915,1558997747,1558997743,20aba14c,41.0492,-73.9115,218,6825,280,⋯,NA,NA,NA,FALSE,FALSE,0,FALSE,FALSE,0,001915_0
,001915,1558997750,1558997752,20aba14c,41.0448,-73.9174,218,7038,280,⋯,NA,NA,NA,FALSE,FALSE,0,FALSE,FALSE,0,001915_0


In [51]:
get_quadrant <- function(azimuth) {
    if(azimuth < 0 | azimuth > 360) {
        stop("Wrong azimuth")
    }
    else if(azimuth >=0 & azimuth < 90) {
        return(1)
    } else if(azimuth >= 90 & azimuth < 180) {
        return(2)
    } else if(azimuth >= 180 & azimuth < 270) {
        return(3)
    } else {
        return(4)
    }
}

get_ac_quadrant <- function(azimuths, ts) {
    quadrants <- sapply(azimuths, get_quadrant)
    durations <- c(0, diff(ts, lag = 1))
    df <- data.frame(quad = quadrants, dur = durations)
    df$dur <- as.numeric(df$dur)
    setDT(df)
    reqd_df <- df[, "total_time" := sum(dur, na.rm = T), by = quad]
    max_records_quad <- table(quadrants)
    max_records_quad <- names(max_records_quad)[which.max(max_records_quad)]
    max_time_quad <- reqd_df$quad[which.max(reqd_df$total_time)]
    if(max_time_quad == as.numeric(max_records_quad)) {
        return(max_time_quad)
    } else {
        return(0)
    }
}

visualize_ac_df <- function(max_ac_df, type = "path") {
    for(id1 in unique(max_ac_df$aircraft)) {
        ac_df <- max_ac_df[max_ac_df$aircraft == id1, ]
        quadrant <- get_ac_quadrant(max_ac_df$azimuth, max_ac_df$ts)
        setwd(as.character(quadrant))
        setorder(ac_df, ts)
        p <- ggplot()
        if(any(ac_df$landed)) {
            landed_rows <- ac_df[ac_df$landed, ]
            for(i in 1:nrow(landed_rows)) {
                p <- p + geom_rect(xmin = landed_rows$lon.y[i] - 0.4, xmax = landed_rows$lon.y[i] + 0.4,
                                  ymin = landed_rows$lat.y[i] - 0.4, ymax = landed_rows$lat.y[i] + 0.4, alpha = 0.2,
                                  fill = "palegreen")
            }
        }
        if(type == "path") {
            p <- p + geom_path(data = ac_df, aes(x = lon.x, y = lat.x, frame = ts, color = altitude),
                               arrow = arrow(angle = 18, length = unit(0.065, "inches"), type = 'closed')) +
                xlim(range_lon.x) +
                scale_colour_gradient2(low = 'black', mid = 'dodgerblue', high = 'orange') +
                ylim(range_lat.x)
        } else {
            p <- p + geom_point(data = ac_df, aes(x = lon.x, y = lat.x, frame = ts, color = altitude), arrow = arrow(angle = 18, length = unit(0.065, "inches"), type = 'closed')) +
                xlim(range_lon.x) +
                scale_colour_gradient2(low = 'black', mid = 'dodgerblue', high = 'orange') +
                ylim(range_lat.x)
        }
        png(width = 880, height = 400, filename = paste0(id1, "_", type, ".png"))
        print(p)
        dev.off()
        setwd("../")
    }
}

visualize_ac_df_points <- function(max_ac_df) {
    visualize_ac_df(max_ac_df, "point")
}

visualize_ac_df_paths <- function(max_ac_df) {
    visualize_ac_df(max_ac_df)
}



In [52]:
library(parallel)
cores <- detectCores() - 1
cl <- makeCluster(cores)
uniq_ac <- unique(max_ac_df$aircraft)
num <- ceiling(length(uniq_ac)/cores)
ac_splits <- factor(ceiling(1:length(uniq_ac)/num))
ac_splits <- split(uniq_ac, ac_splits)
splits <- split(max_ac_df, max_ac_df$aircraft)
splits1 <- lapply(ac_splits, function(ac_split)
    do.call(rbind, splits[ac_split]))
range_lon.x <- range(max_ac_df$lon.x)
range_lat.x <- range(max_ac_df$lat.x)
clusterExport(cl, list("visualize_ac_df", "geom_point", "geom_rect", "max", "unique", "setorder", "paste0", "ggplot", "aes", "geom_path", "arrow", "unit", "xlim", "ylim", "print", "scale_colour_gradient2", "png", "dev.off", "setwd", "get_ac_quadrant", "sapply", "get_quadrant", "data.frame", "as.numeric", "diff", "pi", "table", "names", "which.max", "sum", "setDT", "range_lon.x", "range_lat.x"))


In [53]:
parLapply(cl = cl, X = splits1, fun = visualize_ac_df_paths)



$`1`
NULL

$`2`
NULL

$`3`
NULL

$`4`
NULL

$`5`
NULL

$`6`
NULL

$`7`
NULL

$`8`
NULL

$`9`
NULL

$`10`
NULL

$`11`
NULL

In [54]:
parLapply(cl = cl, X = splits1, fun = visualize_ac_df_points)


$`1`
NULL

$`2`
NULL

$`3`
NULL

$`4`
NULL

$`5`
NULL

$`6`
NULL

$`7`
NULL

$`8`
NULL

$`9`
NULL

$`10`
NULL

$`11`
NULL

In [55]:
stopCluster(cl)

In [ ]:
splits1[[1]]

In [9]:
# func <- function(ac_df, uniq_ts) {
#   for(i_time in uniq_ts) {
#     subset_df <- ac_df[ts <= i_time & i_time <= max_ts, ]
#     p <- ggplot(data = subset_df, aes(x = lon, y = lat, frame = ts)) +
#     geom_path(aes(color = altitude), arrow = arrow(angle = 18, length = unit(0.065, "inches"), type = 'closed')) +
#     xlim(c(min(ac_df$lon), max(ac_df$lon))) +
#     scale_colour_gradient2(low = 'black', mid = 'dodgerblue', high = 'orange') +
#     ylim(c(min(ac_df$lat), max(ac_df$lat)))# +
# #     theme(legend.position = "none")
#     print(p)
#   }
# }

# animate_ac_df <- function(max_ac_df) {
#     for(id1 in unique(max_ac_df$id)) {
#         dir.create(id1)
#         setwd(id1)
#         ac_df <- max_ac_df[max_ac_df$id == id1, ]
#         ac_df$max_ts <- max(ac_df$ts, na.rm = T)
#         uniq_ts <- unique(ac_df$ts)
#         uniq_ts <- uniq_ts[order(uniq_ts)]
#         setorder(ac_df, ts)
#         oopt <- ani.options(interval = 0.05, nmax = nrow(ac_df))
#         saveHTML(func(ac_df, uniq_ts), interval = 0.05, width = 880,
#                  height = 400, htmlfile = paste0(id1, ".html"))
#         setwd("../")
#     }
# }

In [56]:
splits1[[1]][splits1[[1]]$landed, c("aircraft", "lat.y", "lon.y")]

aircraft,lat.y,lon.y
<chr>,<dbl>,<dbl>
0200CA,40.6398,-73.7789
JFK,40.7772,-73.8726
JFK,40.1935,-76.7634
JFK,40.7772,-73.8726
02013D,40.6398,-73.7789
BOS,40.7772,-73.8726
BOS,38.8521,-77.0377
30061E,42.3643,-71.0052
34230A,40.6398,-73.7789


In [10]:
# clusterExport(cl, list("animate_ac_df", "dir.create", "setwd", "max", "unique", "setorder", "ani.options", "saveHTML", "func", "paste0", "ggplot", "aes", "geom_path", "arrow", "unit", "xlim", "ylim", "print", "scale_colour_gradient2"))

In [11]:
# parLapply(cl = cl, X = splits1, fun = animate_ac_df)

In [12]:
# stopCluster(cl)